In [1]:
import keras
import numpy as np

In [2]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


In [3]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [4]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 200278


In [5]:
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

Unique characters: 57


In [6]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
from keras import layers

In [9]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [10]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
import random
import sys

In [14]:
for epoch in range(1, 20):
    print('\nepoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    model.save(f'Text_Model_{epoch}.h5')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
        writer.write(f'GENERATED TEXT FOR EPOCH {epoch}')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('\n------ temperature:', temperature)

        with open(f'Generated_Text_Epoch_{epoch}.txt', 'a+') as writer:
            writer.write('\n-------------------\nTEMPERATURE: ' + str(temperature) + '\n')
            writer.write((generated_text).upper() + '\n\n')

            sys.stdout.write(generated_text.upper())

            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                generated_text += next_char
                generated_text = generated_text[1:]

                sys.stdout.write(next_char)
                writer.write(next_char)

        print(generated_text)
        
model.save('Text_Model.h5')


epoch 1
1565/1565 [==============================] - 197s 126ms/step - loss: 1.4734
--- Generating with seed: "f him!
every hand that lays hold of him shrinks back!--and f"

------ temperature: 0.2
F HIM!
EVERY HAND THAT LAYS HOLD OF HIM SHRINKS BACK!--AND For the sense of the sense of the mastering the conception as a conception and a soul as a soul and the philosople, the sense of the contridive of the sense of the more as a person of the sense of the sense of the same of the sense of the man as a philosoplection of the man as a man and the sense of the sense of the same and something the power of the sense of the sense of the man as a man and discpower of the sense of the sense of the man as a man and disc

------ temperature: 0.5
POWER OF THE SENSE OF THE SENSE OF THE MAN AS A MAN AND DISCordent expecience of the was one any the more the whole because of the the the has precisely the general and man is good of even him as the own characterisy the sense of the german former and mor

<ipython-input-11-66c6eecb696d>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


hers of the sense the constantly and development of the faculty and some nature of the most power, and the f of the faculty and some nature of the most power, and the f

------ temperature: 1.0
 OF THE FACULTY AND SOME NATURE OF THE MOST POWER, AND THE Free specioseizat discovery of the first in uskant to no greally desired acth-is promive
mongqulation of justice. every last vanges of power of himself:
it--as says:a tains clas, and implantion of very cradent of one's
germany
as promatn s
still
things as should faits
alsoh, as the species, he thel weast aider also consisted possseice, is life to their are as
those sone" than
now
power, against wholife to their are as
those sone" than
now
power, against who

------ temperature: 1.2
LIFE TO THEIR ARE AS
THOSE SONE" THAN
NOW
POWER, AGAINST WHO are their sake of in ecrouch."--if, a onger rur.

a oppoest louse? it
has scypateness--does "nlwy formed, they evonce
fa!t so lame contrailly hadrood erjoy of life toess high
dany? that intovely muct